# Assignment 8 - [#1079](https://github.com/alexanderquispe/Diplomado_PUCP/issues/1079)
**Deadline: September 14th - 23:59**

Fernando Mendoza | Andrea Pezo | Michel Cotrina | Estefanny Gil | Armando Ore

## Instructions

1. Get the 15 variables from this raster for all Peru departments polygons. This is the [link](https://github.com/alexanderquispe/Diplomado_PUCP/tree/main/_data/INEI_LIMITE_DEPARTAMENTAL) where shapefiles are located. This is the [link](https://ghsl.jrc.ec.europa.eu/download.php?ds=builtC) of the source raster. The values should be the percentage of district area cover by this specific Morphological Settlement Zone.
2. Then you are going to generate choropleth map using folium for these 15 variables.
3. Save your html in the same folder of your JN. Name your HTML as your branch. This HTML should have all these layers. Please do not forget to use Layer Control.

**Do not upload any other file. Be cautious of uploading files greater than 100MB.**

## Step 0: Library

In [ ]:
#!pip install rasterio numpy matplotlib
#!pip install ipywidgets
#!pip install rasterstats

In [ ]:
import geopandas as gpd
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from shapely.geometry import mapping
import rasterio
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import os
from rasterio.mask import mask
from rasterstats import zonal_stats
import pandas as pd
from shapely.ops import transform

## Step 1:

In [ ]:
# Load your shapefile
shapefile_path = 'INEI_LIMITE_DEPARTAMENTAL.shp'
gdf = gpd.read_file(shapefile_path)
print(gdf.head())

   OBJECTID_1  OBJECTID CCDD   NOMBDEP      CAPITAL  Shape_STAr  Shape_STLe  \
0           1       1.0   01  AMAZONAS  CHACHAPOYAS    3.203006   12.912088   
1           2       2.0   02    ANCASH       HUARAZ    2.954592   11.780424   
2           3       3.0   03  APURIMAC      ABANCAY    1.765933    7.730154   
3           4       4.0   04  AREQUIPA     AREQUIPA    5.330203   17.405040   
4           5       5.0   05  AYACUCHO     AYACUCHO    3.643705   17.127166   

   ORIG_FID  Shape_Leng  Shape_Area                    CORREO  \
0         0   12.912088    3.203006  juan.suyo@geogpsperu.com   
1         1   11.780424    2.954592  juan.suyo@geogpsperu.com   
2         2    7.730154    1.765933  juan.suyo@geogpsperu.com   
3         3   17.405040    5.330203  juan.suyo@geogpsperu.com   
4         4   17.127166    3.643705  juan.suyo@geogpsperu.com   

             CONTACTO   WHATSAPP  \
0  www.geogpsperu.com  931381206   
1  www.geogpsperu.com  931381206   
2  www.geogpsperu.com  931

In [ ]:
# from colab

raster = ["GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10.tif",
          "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11.tif",
          "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C12.tif",
          "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C11.tif",
          "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C12.tif",
          "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C12.tif",
          "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C10.tif",
          "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C11.tif",
          "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C12.tif"
         ]
statistics = []

for raster_path in raster:
    # zonal_stats
    stats = zonal_stats(gdf, raster_path, stats="count sum", categorical=True, all_touched=True)
    # df
    stats_df = pd.DataFrame(stats)
    # concat
    df = pd.concat([gdf.reset_index(drop=True), stats_df], axis=1)
    statistics.append(df)

In [ ]:
# concat all dfs
end_df = pd.concat(statistics, ignore_index=True)
print(end_df.head())
# NaN to 0
end_df.fillna(0, inplace=True)

   OBJECTID_1  OBJECTID CCDD   NOMBDEP      CAPITAL  Shape_STAr  Shape_STLe  \
0           1       1.0   01  AMAZONAS  CHACHAPOYAS    3.203006   12.912088   
1           2       2.0   02    ANCASH       HUARAZ    2.954592   11.780424   
2           3       3.0   03  APURIMAC      ABANCAY    1.765933    7.730154   
3           4       4.0   04  AREQUIPA     AREQUIPA    5.330203   17.405040   
4           5       5.0   05  AYACUCHO     AYACUCHO    3.643705   17.127166   

   ORIG_FID  Shape_Leng  Shape_Area                    CORREO  \
0         0   12.912088    3.203006  juan.suyo@geogpsperu.com   
1         1   11.780424    2.954592  juan.suyo@geogpsperu.com   
2         2    7.730154    1.765933  juan.suyo@geogpsperu.com   
3         3   17.405040    5.330203  juan.suyo@geogpsperu.com   
4         4   17.127166    3.643705  juan.suyo@geogpsperu.com   

             CONTACTO   WHATSAPP  \
0  www.geogpsperu.com  931381206   
1  www.geogpsperu.com  931381206   
2  www.geogpsperu.com  931

In [1]:
!pip install folium geopandas


In [2]:
import folium
import geopandas as gpd


In [4]:
import geopandas as gpd

df = gpd.read_file('INEI_LIMITE_DEPARTAMENTAL.shp')
df.head()

,OBJECTID_1,OBJECTID,CCDD,NOMBDEP,CAPITAL,Shape_STAr,Shape_STLe,ORIG_FID,Shape_Leng,Shape_Area,CORREO,CONTACTO,WHATSAPP,geometry
0,1,1.0,01,AMAZONAS,CHACHAPOYAS,3.203006,12.912088,0,12.912088,3.203006,juan.suyo@geogpsperu.com,www.geogpsperu.com,931381206,"POLYGON ((-77.81399 -2.99278, -77.81483 -2.995..."
1,2,2.0,02,ANCASH,HUARAZ,2.954592,11.780424,1,11.780424,2.954592,juan.suyo@geogpsperu.com,www.geogpsperu.com,931381206,"POLYGON ((-77.64697 -8.05086, -77.64689 -8.051..."
2,3,3.0,03,APURIMAC,ABANCAY,1.765933,7.730154,2,7.730154,1.765933,juan.suyo@geogpsperu.com,www.geogpsperu.com,931381206,"POLYGON ((-73.74655 -13.17442, -73.74570 -13.1..."
3,4,4.0,04,AREQUIPA,AREQUIPA,5.330203,17.405040,3,17.405040,5.330203,juan.suyo@geogpsperu.com,www.geogpsperu.com,931381206,"POLYGON ((-71.98109 -14.64062, -71.98093 -14.6..."
4,5,5.0,05,AYACUCHO,AYACUCHO,3.643705,17.127166,4,17.127166,3.643705,juan.suyo@geogpsperu.com,www.geogpsperu.com,931381206,"POLYGON ((-74.34843 -12.17503, -74.35000 -12.1..."


In [8]:
import geopandas as gpd


gdf = gpd.read_file('INEI_LIMITE_DEPARTAMENTAL.shp')

print(gdf.head())


   OBJECTID_1  OBJECTID CCDD   NOMBDEP      CAPITAL  Shape_STAr  Shape_STLe  \
0           1       1.0   01  AMAZONAS  CHACHAPOYAS    3.203006   12.912088   
1           2       2.0   02    ANCASH       HUARAZ    2.954592   11.780424   
2           3       3.0   03  APURIMAC      ABANCAY    1.765933    7.730154   
3           4       4.0   04  AREQUIPA     AREQUIPA    5.330203   17.405040   
4           5       5.0   05  AYACUCHO     AYACUCHO    3.643705   17.127166   

   ORIG_FID  Shape_Leng  Shape_Area                    CORREO  \
0         0   12.912088    3.203006  juan.suyo@geogpsperu.com   
1         1   11.780424    2.954592  juan.suyo@geogpsperu.com   
2         2    7.730154    1.765933  juan.suyo@geogpsperu.com   
3         3   17.405040    5.330203  juan.suyo@geogpsperu.com   
4         4   17.127166    3.643705  juan.suyo@geogpsperu.com   

             CONTACTO   WHATSAPP  \
0  www.geogpsperu.com  931381206   
1  www.geogpsperu.com  931381206   
2  www.geogpsperu.com  931

In [9]:
print(gdf.columns)


Index(['OBJECTID_1', 'OBJECTID', 'CCDD', 'NOMBDEP', 'CAPITAL', 'Shape_STAr',
       'Shape_STLe', 'ORIG_FID', 'Shape_Leng', 'Shape_Area', 'CORREO',
       'CONTACTO', 'WHATSAPP', 'geometry'],
      dtype='object')


In [10]:
print(gdf.crs)


EPSG:4326


In [12]:
!pip install rasterstats


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 9.0 MB/s eta 0:00:00


In [13]:
from rasterstats import zonal_stats
import pandas as pd


In [14]:
raster = [
    "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10.tif",
    "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11.tif",
    "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C12.tif",
    "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C11.tif",
    "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C12.tif",
    "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C12.tif",
    "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C10.tif",
    "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C11.tif",
    "GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C12.tif"
]

In [15]:
#list to save results
statistics = []

In [19]:
for i, raster_path in enumerate(raster):
    # zonal statistics
    stats = zonal_stats(gdf, raster_path, stats="sum", categorical=True, all_touched=True)
    stats_df = pd.DataFrame(stats)

    # identify each raster
    stats_df.columns = [f'Variable_{i+1}_{col}' for col in stats_df.columns]

    # combine statistics with GeoDataFrame
    gdf = pd.concat([gdf.reset_index(drop=True), stats_df], axis=1)  # Corregida la indentación aquí

# check results
print(gdf.head())

   OBJECTID_1  OBJECTID CCDD   NOMBDEP      CAPITAL  Shape_STAr  Shape_STLe  \
0           1       1.0   01  AMAZONAS  CHACHAPOYAS    3.203006   12.912088   
1           2       2.0   02    ANCASH       HUARAZ    2.954592   11.780424   
2           3       3.0   03  APURIMAC      ABANCAY    1.765933    7.730154   
3           4       4.0   04  AREQUIPA     AREQUIPA    5.330203   17.405040   
4           5       5.0   05  AYACUCHO     AYACUCHO    3.643705   17.127166   

   ORIG_FID  Shape_Leng  Shape_Area  ...  \
0         0   12.912088    3.203006  ...   
1         1   11.780424    2.954592  ...   
2         2    7.730154    1.765933  ...   
3         3   17.405040    5.330203  ...   
4         4   17.127166    3.643705  ...   

                                            geometry Variable_1_sum  \
0  POLYGON ((-77.81399 -2.99278, -77.81483 -2.995...           None   
1  POLYGON ((-77.64697 -8.05086, -77.64689 -8.051...           None   
2  POLYGON ((-73.74655 -13.17442, -73.74570 -13

In [22]:
import folium

# to creat the map
m = folium.Map(location=[-9.19, -75.015], zoom_start=5)



In [27]:
# Select only the columns that are numeric and avoid the geometry column
numeric_columns = gdf.select_dtypes(include=['float64', 'int64']).columns  # Select only numeric columns

# Convert all variable columns to numeric, forcing errors to NaN
for column in numeric_columns:  # Iterate over numeric columns only
    gdf[column] = pd.to_numeric(gdf[column], errors='coerce')  # Convert to numeric and coerce errors to NaN

# Fill NaN values with 0 or another appropriate value, only for numeric columns
gdf[numeric_columns] = gdf[numeric_columns].fillna(0)  # Fill NaN values in numeric columns


In [28]:
# Iterate over the columns of variables and add a choropleth layer for each one
for column in numeric_columns:  # Iterate over cleaned numeric columns
    folium.Choropleth(
        geo_data=gdf,
        name=column,
        data=gdf,
        columns=['NOMBDEP', column],  # 'NOMBDEP' is the department name column in the GeoDataFrame
        key_on='feature.properties.NOMBDEP',  # Key to match data and geometry
        fill_color='YlOrRd',  # Color scheme for the layer
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f'Cobertura de {column}'  # Legend for the layer
    ).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Save the map as HTML
m.save('andreapezonu_mapa.html')  # Change the name as needed

In [29]:
import shutil

# this is to make the file smaller
shutil.make_archive('andreapezonu_mapa', 'zip', '.', 'andreapezonu_mapa.html')


'/content/andreapezonu_mapa.zip'